In [1]:
#Cab visualization but with sensors in NYC

In [2]:
import pandas as pd
import numpy as np
import time 

import folium
from folium.plugins import HeatMap
from IPython.display import IFrame

import os
import time
from selenium import webdriver
from PIL import Image 
import glob

In [3]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

In [4]:
sensors = pd.read_csv('Sensors_NYC.csv')
sensors.head()

,latitude,longitude
0,40.733326,-73.991972
1,40.597676,-73.964833
2,40.593135,-73.969560
3,40.715699,-73.803864
4,40.725978,-73.723547


In [5]:
df_train = pd.read_csv('NYC-Cab-Pickup_train.csv')
df_train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,3/14/2016 17:24,3/14/2016 17:32,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,6/12/2016 0:43,6/12/2016 0:54,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,1/19/2016 11:35,1/19/2016 12:10,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,4/6/2016 19:32,4/6/2016 19:39,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,3/26/2016 13:30,3/26/2016 13:38,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [10]:
#getting map center
def get_center_latlong(sensors):
    # get the center of my map for plotting
    centerlat = (sensors['latitude'].max() + sensors['latitude'].min()) / 2
    centerlong = (sensors['longitude'].max() + sensors['longitude'].min()) / 2
    return centerlat, centerlong

In [7]:
#Data preprocessing to get the hour and day of week for all entries in my training set
df = df_train[['pickup_datetime', 'pickup_longitude', 'pickup_latitude']].copy()
df['day_of_week'] = pd.to_datetime(df['pickup_datetime']).dt.dayofweek
df['hour_of_day'] = pd.to_datetime(df['pickup_datetime']).dt.hour
df.head()

,pickup_datetime,pickup_longitude,pickup_latitude,day_of_week,hour_of_day
0,3/14/2016 17:24,-73.982155,40.767937,0,17
1,6/12/2016 0:43,-73.980415,40.738564,6,0
2,1/19/2016 11:35,-73.979027,40.763939,1,11
3,4/6/2016 19:32,-74.010040,40.719971,2,19
4,3/26/2016 13:30,-73.973053,40.793209,5,13


In [8]:
# Function to visualize map
def embed_map(map, filename):
    map.save(filename)
    return IFrame(filename, width='100%', height='500px')

In [11]:
#Plot heatmap of lat/long coordinates by hour and day of week

dow_dict = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
center = get_center_latlong(sensors)

In [18]:
# Instantiate map object 
map_5 = folium.Map(location=[40.75, -73.96], tiles='openstreetmap', zoom_start=12)
        
        # go through each home in set, make circle, and add to map.
for i in range(len(sensors)):
    folium.Circle(
                    location=[sensors.iloc[i]['latitude'], sensors.iloc[i]['longitude']],
                    radius=10,
            ).add_to(map_5)

In [19]:
for i in range(df.day_of_week.min(), df.day_of_week.max()+1):
    for j in range(df.hour_of_day.min(), df.hour_of_day.max()+1): 
        
            # Filter to include only data for each day of week and hour of day
            df_geo = df.loc[(df.day_of_week==i) & (df.hour_of_day==j)][['pickup_latitude', 'pickup_longitude']].copy()
            
            # Plot heatmap
            HeatMap(data=df_geo, radius=10).add_to(map_5)

            # Get day of week string from dow_dict
            d = dow_dict.get(i)
        
            # Add title to heatmap
            title_html = f'''<h3 align="center" style="font-size:20px">
                        <b>CO2 emissions at New York City on </b></h3>
                     '''
            map_5.get_root().html.add_child(folium.Element(title_html))
            
            # Save map
            embed_map(map_5, f'./NYC_html_maps/{i}_{j}_heatmap.html')

In [20]:
#Automate Screenshots of heatmap files using selenium

for i in range(0, 7):
    for j in range(0, 24):
        # Set file path
        tmpurl= f'C:/Users/Nivashini/Desktop/Desktop/Ribbit Network/NYC_html_maps/{i}_{j}_heatmap.html'
        
        # Set browser to Chrome
        browser = webdriver.Chrome()
        
        # Open file in browser
        browser.get(tmpurl)
            
        # If hour is < 10, add 0 for sorting purposes and to keep chronological order
        if j < 10:
            browser.save_screenshot(f'./maps_png_pickup/{i}_0{j}_heatmap.png')
        else:
            browser.save_screenshot(f'./maps_png_pickup/{i}_{j}_heatmap.png')
        
        # Close browser
        browser.quit()

In [21]:
#Create Animated .gif file

def png_to_gif(path_to_images, save_file_path, duration=500):
    frames = []
    
    # Retrieve image files
    images = glob.glob(f'{path_to_images}')
    
    # Loop through image files to open, resize them and append them to frames
    for i in sorted(images): 
        im = Image.open(i)
        im = im.resize((550,389),Image.ANTIALIAS)
        frames.append(im.copy())
        
    # Save frames/ stitched images as .gif
    frames[0].save(f'{save_file_path}', format='GIF', append_images=frames[1:], save_all=True,
                   duration=duration, loop=0)

In [22]:
png_to_gif(path_to_images='./maps_png_pickup/*.png', 
           save_file_path='./plots/pickup_heatmap.gif',
          duration = 500)

C:\Users\Nivashini\AppData\Local\Temp\ipykernel_39976\1078933910.py:12: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  im = im.resize((550,389),Image.ANTIALIAS)
